# Feature Extraction

## Import libraries

In [31]:
from typing import Tuple
from time import sleep
import time
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from os import path
from subprocess import Popen, call
import subprocess
import pyautogui
import ffmpeg
import signal
import os
import re
import pandas as pd

In [27]:
from PIL import Image
from pytesseract import pytesseract

In [ ]:
image_path='./extracted/bfbf7a2b7ac635e67877b1ab87fd6629-b7a604b3b08f0862ef0e_start_frame.png'
img = Image.open(image_path)
pytesseract.tesseract_cmd = path_to_tesseract
text = pytesseract.image_to_string(img)

In [42]:
def get_instruction(image_path):
    path_to_tesseract = r"/usr/bin/tesseract"
    img = Image.open(image_path)
    pytesseract.tesseract_cmd = path_to_tesseract
    text = pytesseract.image_to_string(img)
    text = text.lower()
    print(text.lower())
    print("Detected Text: \n" + text)
    if 'tap' in text:
        return 'tap'
    elif 'swipe l' in text:
        return'swipe left'
    elif 'swipe r' in text:
        return 'swipe right'
    elif 'swipe u' in text:
        return 'swipe up'
    elif 'swipe do' in text:
        return 'swipe down'
    elif 'swipe' in text:
        return 'swipe'
    else:
        return 'tap_try'

In [43]:
pwd

'/home/owon/Documents/10x/week11/notebooks'

In [44]:
get_instruction('459c60702cfaae32207a161b693df47c_start_frame.png')

In [37]:
print(text_12.replace('\n', ' ') , "------\n", text_13.replace('\n', ' '), "----\n", text_14.replace('\n', ' '))

ovexus l/certified by lexus  philadelphia area 2 lexus dealers a=.  learn more  ------
 ovexxuss l/certified by lexus  philadelphia area _ lexusdealers <7  pe  lartified  learn more   ----
 ovexus l/certified by lexus  philadelphia area _ lexus dealers —  l/certified  lexus  learn more 


In [47]:
image_path = '../start_frame/459c60702cfaae32207a161b693df47c_start_frame.jpg'
path_to_tesseract = r"/usr/bin/tesseract"
img = Image.open(image_path)
pytesseract.tesseract_cmd = path_to_tesseract
text_13 = pytesseract.image_to_string(img).lower()
print(text_13)

In [61]:
now = time.time()

## read the creative

In [11]:
class creativeFrameExtractor:
    '''
    class responsible for Extracting Creative Start and End Frames. It requires a chrome webdriver compatible with 
    selenium to be installed/ included in the run environment path.
    '''

    def __init__(self, preview_url: str,
                 save_location: str = '',
                 browser_edges: Tuple[float, float] = (70, 1039)) -> None:
        self.preview_url = preview_url
        self.start_time = time.time()
        self.engaged = False
        self.engagement_type = 'tap'
        self.browser_edges = browser_edges
        self.file_name = '-'.join(preview_url.split('/')[-3:-1])
        self.save_location = save_location
        self.video_name = path.join(self.save_location, self.file_name)
        self.cmd = f" ffmpeg -f alsa -ac 2 -i pulse -video_size 1920x1080 -framerate 60 -f x11grab -i :1 -c:v libx264rgb -qp 0 -preset ultrafast {self.video_name}.mkv -y"

        # Config

        # Browser config
        # Browser Options
        self.opt = Options()
        self.opt.add_argument("--hide-scrollbars")
        self.opt.add_experimental_option(
            "excludeSwitches", ["enable-automation"]
        )
        # Browser logs
        self.capabilities = DesiredCapabilities.CHROME
        self.capabilities["goog:loggingPrefs"] = {"browser": "ALL"}

    def is_status_complete(self, passed_driver) -> bool:
        '''
        Function to check if the Ad-Unit is complete
        '''
        # Retrieve logs from the browser
        logs = passed_driver.get_log("browser")
        for log in logs:
            # select logs coming from Ad-Unit
            if log["source"] == "console-api":
                # Extract message from log
                message = log["message"].lower()

                if '"GAME CREATED"'.lower() in message or '"DROPPED"'.lower() in message:
                    # Start recording Game
                    print("Starting Recording Ad-Unit...")

                    return False
                if '"START"'.lower() in message:
                    self.engaged = True
                    # Engaged
                    print("Ad-UNIT Engaged...")
                    return False
                
                # duration = time.time() - self.start_time
                if '"START"'.lower() not in message:
                    print("Ad-Unit not engaged !!!")

                    return True
                # if '"START"'.lower() in message:
                #     print("Wrong Ad-Unit Engagement !!!")

                    return True
                if '"GAME COMPLETE"'.lower() in message or '"video-top-ENDED"'.lower() in message or '"win"'.lower() in message:
                    # Stop Recording Game
                    print("Stopped Recording Ad-Unit...")
                    print(message)
                    return True
        return False

    @staticmethod
    def terminate(process: call) -> None:  # call[bytes]
        '''
        Function to terminate a process
        '''
        # Video recording process terminator
        if process.poll() is None:
            call("kill -6 " + str(process.pid),
                 stdin=subprocess.PIPE, shell=True)

    @staticmethod
    def crop_video(filename: str, x_pos: float = 0, y_pos: float = 200, width: float = 650, height: float = 900) -> None:
        '''
        Function to crop a video with a location and size parameters
        '''
        print(filename)
        input_video = ffmpeg.input(f"{filename}.mkv")
        cropped_video = ffmpeg.crop(input_video, x_pos, y_pos, width, height)
        output_video = ffmpeg.output(cropped_video, f"{filename}_cropped.mkv")
        ffmpeg.run(output_video)

    @staticmethod
    def get_audio(filename: str):
        cmd_get_audio = f" ffmpeg -i {filename}.mkv -q:a 0 -map a {filename}.mp3 -y"
        fetch_audio = Popen(cmd_get_audio, stdout=subprocess.PIPE,
                            shell=True, preexec_fn=os.setsid)
        exitcode = fetch_audio.wait()

    def _imitate_engagement(self, ad_size: Tuple[float, float]) -> None:
        '''
        Function to imitate a given engagement type.
        '''
        center = (ad_size[0]/2, self.browser_edges[0]+(ad_size[1]/2))

        if self.engagement_type == "tap":
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.leftClick()
        elif self.engagement_type == "swipe right":
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.dragRel(center[0], 0, duration=1)
        elif self.engagement_type == "swipe left" or self.engagement_type == 'swipe':
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.dragRel(-center[0], 0, duration=1)
        elif self.engagement_type == "swipe down":
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.dragRel(0, center[1], duration=1)
        elif self.engagement_type == "swipe up":
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.dragRel(0, -center[1], duration=1)
        elif self.engagement_type == "tap and hold":
            pyautogui.moveTo(center[0], center[1], duration=1)
            pyautogui.click()
        elif self.engagement_type == "scrub":
            pyautogui.moveTo(center[0] - (1/2 * center[0]),
                             center[1] - (2/3 * center[1]), duration=0.2)

            pyautogui.dragRel(center[0], 0, duration=0.2)
            pyautogui.dragRel(-center[0], (1/3 * center[1]), duration=0.2)
            pyautogui.dragRel(center[0], 0, duration=0.2)
            pyautogui.dragRel(-center[0], (1/3 * center[1]), duration=0.2)
            pyautogui.dragRel(center[0], 0, duration=0.2)

    def generate_preview_video(self, links: list, bad: list) -> None:
        '''
        Function to generate preview video and also a cropped version of the video
        '''
        # initialize selenium webDriver
        driver = webdriver.Chrome("/usr/bin/chromedriver",
                                  options=self.opt, desired_capabilities=self.capabilities)
        # maximize webdriver's window to maximize size
        driver.maximize_window()
        # get the starting frame
        url_not_working = []
        
        links = [x for x in links if x not in bad]
        for link in links:
            file_name = '-'.join(link.split('/')[-3:-2])

            try:
                # Load Ad-Unit through selenium
                driver.get(link)

                # Locate Ad-Unit Element from Browser
                canvas = driver.find_element(By.TAG_NAME, "canvas")

                # Capture start frame
                canvas.screenshot(
                    path.join(
                        './extracted/',  f'{file_name}_start_frame.png')
                )

            except TimeoutException:
                print("TimeoutException Throwed")
                print("Ad-Unit Status console logs did not complete. Engagement Failed.")
                url_not_working.append(link)
                # driver.close()

            except NoSuchElementException:
                print(f"Ad-Unit Failed to load: {i}")
                url_not_working.append(link)

            # record the video

            try:
                # # load Ad-Unit with selenium
                # driver.get(self.preview_url)

                engagement_type = get_instruction(
                    './extracted/' + f'{file_name}_start_frame.png')

                print("Engagement Chosen:" + self.engagement_type)

                # locate Ad-Unit element from browser
                
                # canvas = driver.find_element(By.TAG_NAME, "canvas")

                self.cmd = f" ffmpeg -hide_banner -loglevel error -f alsa -ac 2 -i pulse -video_size 1920x1080 -framerate 60 -f x11grab -i :1 -c:v libx264rgb -qp 0 -preset ultrafast ./extracted/{file_name}.mkv -y"

                

                # start recording the entire screen
                video_recording = Popen(
                    self.cmd, stdout=subprocess.PIPE, shell=True, preexec_fn=os.setsid)

                # Identify the size of Ad-Unit
                ad_size = (canvas.size.get("width"), canvas.size.get("height"))

                # Engage Ad Unite
                self._imitate_engagement(ad_size)

                self.start_time = time.time()
                # Continuously check Status of Ad-Unit using it's console log
                # until it reached a "GAME COMPLETE" Status
                # driver.get_log("browser")["message"]
                WebDriverWait(driver, 45).until(self.is_status_complete or (time.time() - self.start_time) > 45)
                print(f"Elapsed time:{time.time() - self.start_time}")
                sleep(2)

                canvas.screenshot(
                    path.join(
                        './new_extracted/' + f'{file_name}_end_frame.png')
                )
                print('End frame Captured')

                # stop video recording
                os.killpg(os.getpgid(video_recording.pid), signal.SIGTERM)

                sleep(3)
                # os.kill(video_recording.pid, signal.SIGKILL)

                # os.system(f"kill -9 {video_recording.pid}")

                # close selenium Browser window
                # driver.close()

                # crop Generated Preview video recording
                # self.crop_video(self.video_name, x_pos=0, y_pos=70,
                #                 width=ad_size[0], height=ad_size[1])

                # get audio
                # self.get_audio(self.video_name)

            except TimeoutException:
                print("TimeoutException Throwed")
                print("Ad-Unit Status console logs did not complete. Engagement Failed.")
                url_not_working.append(link)

                # driver.close()

            except NoSuchElementException:
                print(f"Ad-Unit Failed to load: {self.preview_url}")
                url_not_working.append(link)

        driver.close()
        pd.DataFrame.from_dict({
            'url_not_working': url_not_working
        }).to_csv('./url_not_working.csv', index=False)

    def generate_start_frame(self) -> None:
        '''
        function to generate creative start and end frames.
        '''
        # Initialize selenium driver
        driver = webdriver.Chrome(
            options=self.opt, desired_capabilities=self.capabilities,
        )

        # Maximize webdriver window for maximum resolution
        driver.maximize_window()

        try:
            # Load Ad-Unit through selenium
            driver.get(self.preview_url)

            # Locate Ad-Unit Element from Browser
            canvas = driver.find_element(By.TAG_NAME, "canvas")

            # Capture start frame
            canvas.screenshot(
                path.join(self.save_location,
                          f'{self.file_name}_start_frame.png')
            )
            print('Start frame Captured')

            #

            # close the selenium browser window
            driver.close()
        except TimeoutException:
            print("TimeoutException Throwed")
            print("Ad-Unit Status console logs did not complete. Engagement Failed.")
            driver.close()

        except NoSuchElementException:
            print(f"Ad-Unit Failed to load:{self.preview_url}")
            driver.close()

    def generate_frames(self) -> None:
        '''
        function to generate creative start and end frames.
        '''
        # Initialize selenium driver
        driver = webdriver.Chrome(
            options=self.opt, desired_capabilities=self.capabilities,
        )

        # Maximize webdriver window for maximum resolution
        driver.maximize_window()

        try:
            # Load Ad-Unit through selenium
            driver.get(self.preview_url)

            # Locate Ad-Unit Element from Browser
            canvas = driver.find_element(By.TAG_NAME, "canvas")

            # Capture start frame
            canvas.screenshot(
                path.join(self.save_location,
                          f'{self.file_name}_start_frame.png')
            )
            print('Start frame Captured')

            # Identify size of Ad-Unit
            ad_size = (canvas.size.get("width"), canvas.size.get("height"))

            # Engage Ad-Unit
            self._imitate_engagement(ad_size)

            # Continuously check Status of Ad-Unit using it's console log
            # until it reached a "GAME COMPLETE" Status
            WebDriverWait(driver, 100).until(
                self.is_status_complete or self.start_time < 60)

            sleep(5)

            # capture End frame
            canvas.screenshot(
                path.join(self.save_location,
                          f'{self.file_name}_end_frame.png')
            )
            print('End frame Captured')

            # close the selenium browser window
            driver.close()
        except TimeoutException:
            print("TimeoutException Throwed")
            print("Ad-Unit Status console logs did not complete. Engagement Failed.")
            driver.close()

        except NoSuchElementException:
            print(f"Ad-Unit Failed to load: {self.preview_url}")
            driver.close()


In [13]:
url = ""
ext = creativeFrameExtractor(url, save_location = './extracted/')

In [15]:
bad_url = [] 


In [16]:
ext.generate_preview_video(links, bad_url)

/home/owon/anaconda3/envs/newenv/lib/python3.7/site-packages/ipykernel_launcher.py:140: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


Engagement Chosen:tap
Starting Recording Ad-Unit...
Starting Recording Ad-Unit...
Ad-UNIT Engaged...
Ad-Unit not engaged !!!
Elapsed time:11.260973453521729
End frame Captured
Engagement Chosen:tap
Ad-Unit not engaged !!!
Elapsed time:0.007070779800415039
End frame Captured
Engagement Chosen:tap
Ad-UNIT Engaged...
Ad-Unit not engaged !!!
Elapsed time:0.5180180072784424
End frame Captured
Engagement Chosen:tap
Ad-UNIT Engaged...
Ad-Unit not engaged !!!
Elapsed time:3.0992937088012695
End frame Captured
Engagement Chosen:tap
Ad-UNIT Engaged...
Ad-Unit not engaged !!!
Elapsed time:1.0280187129974365
End frame Captured


# get starting frame for all directories

In [281]:
 def generate_start_frame(links: list) -> None:
        '''
        function to generate creative start and end frames.
        '''
        browser_edges =  (70, 1039)
        count =0
        
        opt = Options()
        opt.add_argument("--hide-scrollbars")
        opt.add_experimental_option(
            "excludeSwitches", ["enable-automation"]
        )
        # Browser logs
        capabilities = DesiredCapabilities.CHROME
        capabilities["goog:loggingPrefs"] = {"browser": "ALL"}
        
        # Initialize selenium driver
        driver = webdriver.Chrome(
            options=opt,desired_capabilities = capabilities,
        )
        
        #Maximize webdriver window for maximum resolution
        driver.maximize_window()
        for i in links:
            file_name = '-'.join(i.split('/')[-3:-2])
            
            try:
                # Load Ad-Unit through selenium
                driver.get(i)
                
                # Locate Ad-Unit Element from Browser
                canvas = driver.find_element(By.TAG_NAME,"canvas")
                
                # Capture start frame
                canvas.screenshot(
                    path.join('./new_extracted/' f'{file_name}_start_frame.png')
                )
                count +=1
                            
                #  
                
                # close the selenium browser window
                # driver.close()
            except TimeoutException:
                print("TimeoutException Throwed")
                print("Ad-Unit Status console logs did not complete. Engagement Failed.")
                # driver.close()
            
                
            except NoSuchElementException:
                print(f"Ad-Unit Failed to load: {i}")
                # driver.close()
        driver.close()
        print(count)

In [283]:
generate_start_frame(links)

Start frame Captured
Start frame Captured
Start frame Captured
Start frame Captured
Start frame Captured
Start frame Captured
Start frame Captured
Start frame Captured
Start frame Captured
Start frame Captured
Start frame Captured
Start frame Captured


In [123]:
import pandas as pd

In [170]:
df = pd.read_csv("../data/performance_data.csv")

In [174]:
df['preview'] = df['game_id'].str.split(',') 

In [175]:
ss = df.preview

In [176]:
ss[2]

['e68e20f592457b875ce29757ab855dfe']

In [178]:
len(ss)

905

In [180]:
def create_dirs(ss):
    for i in range(len(ss)):
        directory = './extracted/'+ ss[i][0]
        if not os.path.exists(directory):
            os.makedirs(directory)

In [181]:
create_dirs(ss)


In [19]:
ext.video_name

'./extracted/bfbf7a2b7ac635e67877b1ab87fd6629-b7a604b3b08f0862ef0e'

## Audio Transcribing

In [ ]:
!pip install git+https://github.com/openai/whisper.git


In [ ]:
!pip install setuptools-rust

In [ ]:
import whisper

### Approach One

In [63]:
audio_path = ext.video_name + '.mp3'

In [64]:
model = whisper.load_model('base')
result = model.transcribe(audio_path)

100%|████████████████████████████████████████| 139M/139M [02:35<00:00, 934kiB/s]


In [65]:
result['text']

" Can't take my eyes off of you. Fall in love. With the highly awarded, El certified by Lex's collection of pre-owned vehicles. Exclusively are your Lex's dealer."

### Approach Two

In [67]:
model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio(audio_path)
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: en
I'm sorry.
